# This notebook reads in TomTom raw data and prepares data for conflation

In [1]:
import fiona
import geopandas as gpd
import pandas as pd

# reading tomtom data

In [3]:
# loading tomtom data source

gdb_file = "../data/external/tomtom/Network_region.gdb"

# Get all the layers from the .gdb file 

layers = fiona.listlayers(gdb_file)

In [4]:
# street link layer

link_gdf = gpd.read_file(gdb_file, layer = 'mn_nw_BayArea')

In [5]:
# street link attributes

link_gdf.columns

Index(['ID', 'FEATTYP', 'FT', 'F_JNCTID', 'F_JNCTTYP', 'T_JNCTID', 'T_JNCTTYP',
       'PJ', 'METERS', 'FRC', 'NETCLASS', 'NETBCLASS', 'NET2CLASS', 'NAME',
       'NAMELC', 'SOL', 'NAMETYP', 'CHARGE', 'SHIELDNUM', 'RTETYP', 'RTEDIR',
       'RTEDIRVD', 'PROCSTAT', 'FOW', 'SLIPRD', 'FREEWAY', 'BACKRD', 'TOLLRD',
       'RDCOND', 'STUBBLE', 'PRIVATERD', 'CONSTATUS', 'ONEWAY', 'F_BP', 'T_BP',
       'F_ELEV', 'T_ELEV', 'KPH', 'MINUTES', 'POSACCUR', 'CARRIAGE', 'LANES',
       'RAMP', 'ADA', 'TRANS', 'DYNSPEED', 'SPEEDCAT', 'NTHRUTRAF', 'ROUGHRD',
       'PARTSTRUC', 'ORDER08', 'Shape_Length', 'geometry'],
      dtype='object')

In [6]:
# convert to ESPG lat-lon

link_gdf = link_gdf.to_crs({'init': 'epsg:4326'})
link_gdf.crs

{'init': 'epsg:4326'}

# there is no existing unique tomtom handle for Bay Area, thus we need to create unique handle

In [7]:
# total # tomtom links

link_gdf.shape

(602060, 53)

In [8]:
# unique tomtom handles

len(link_gdf.groupby(["ID", "F_JNCTID", "T_JNCTID"]).count())

601846

In [9]:
# generating unique handle for tomtom

link_gdf["tomtom_link_id"] = range(1, len(link_gdf)+1)

In [11]:
link_gdf[["tomtom_link_id", "geometry"]].iloc[1*10 :(1+1)*10]

,tomtom_link_id,geometry
10,11,(LINESTRING (-123.0221169995461 38.80109000009...
11,12,(LINESTRING (-123.0142609997229 38.80494599934...
12,13,(LINESTRING (-123.0170649992541 38.81000799984...
13,14,(LINESTRING (-122.7365599998377 38.34300099973...
14,15,(LINESTRING (-122.6129169998391 38.34361200020...
15,16,(LINESTRING (-122.7844010002056 38.34604299989...
16,17,(LINESTRING (-122.7647339995357 38.34587100021...
17,18,(LINESTRING (-122.6910070004136 38.34813400042...
18,19,(LINESTRING (-122.6989400002297 38.34855299937...
19,20,(LINESTRING (-123.0606939995791 38.35021500024...


In [14]:
link_gdf.LANES.value_counts()

0     448466
2      66145
3      34110
1      21904
4      20391
5       9004
6       1682
7        321
8         21
9          9
10         5
17         2
Name: LANES, dtype: int64

# Partition tomtom by county for shst match

In [22]:
for i in range(14):
    boundary_gdf = gpd.read_file("../data/external/county_boundaries/boundary_"+str(i+1)+".geojson")
    
    sub_tomtom_gdf = link_gdf[link_gdf.intersects(boundary_gdf.geometry.unary_union)].copy()
    
    sub_tomtom_gdf[["tomtom_link_id", "geometry"]].to_file("../data/external/tomtom/tomtom"+str(i+1)+".in.geojson",
                                                driver = "GeoJSON")

In [37]:
link_gdf.head(3)

,ID,FEATTYP,FT,F_JNCTID,F_JNCTTYP,T_JNCTID,T_JNCTTYP,PJ,METERS,FRC,...,TRANS,DYNSPEED,SPEEDCAT,NTHRUTRAF,ROUGHRD,PARTSTRUC,ORDER08,Shape_Length,geometry,tomtom_link_id
0,6.840000e+13,4110,0,6.840001e+13,0,6.840001e+13,0,0,1554.62,7,...,0,0,7,0,0,0,CA097,1989.817478,(LINESTRING (-122.8315739998535 38.62133700023...,1
1,6.840000e+13,4110,0,6.840001e+13,0,6.840001e+13,0,0,551.09,7,...,0,0,6,0,0,0,CA097,704.628862,(LINESTRING (-122.83972099993 38.6229560004714...,2
2,6.840000e+13,4110,0,6.840001e+13,0,6.840002e+13,0,0,53.22,7,...,0,0,7,0,0,0,CA097,68.054041,(LINESTRING (-122.868480999434 38.626759999424...,3


In [34]:
link_gdf[~(link_gdf.geom_type == 'MultiLineString')]

,ID,FEATTYP,FT,F_JNCTID,F_JNCTTYP,T_JNCTID,T_JNCTTYP,PJ,METERS,FRC,...,TRANS,DYNSPEED,SPEEDCAT,NTHRUTRAF,ROUGHRD,PARTSTRUC,ORDER08,Shape_Length,geometry,tomtom_link_id


# SF county, combination of ["ID", "F_JNCTID", "T_JNCTID"] can serve as unique handle

In [24]:
# read county boundry

sf_boundry = gpd.read_file("../external_data/county/boundary_4.geojson")
sf_boundry.crs

{'init': 'epsg:4326'}

In [43]:
# tomtom links in sf
sf_link_gdf = link_gdf[link_gdf.intersects(sf_boundry.geometry.unary_union)].copy()

In [39]:
sf_link_gdf.shape

(31805, 54)

In [46]:
len(sf_link_gdf.groupby(["ID", "F_JNCTID", "T_JNCTID"]).count())

31805

In [44]:
# write out complete tomtom atrributes in geojson

sf_link_gdf.to_file("../tomtom/tomtom_sf.geojson", driver = "GeoJSON")

In [ ]:
# write out sf tomtom for SHST match

sf_link_gdf[["ID", "F_JNCTID", "T_JNCTID", "geometry"]].to_file("../tomtom/Network_region_shape/tomtom_sf.in.geojson",
                                                driver = "GeoJSON")